In [41]:
%matplotlib inline
import matplotlib.pyplot as plt
import urllib, sys, bs4
import pandas as pd
from collections import OrderedDict
from operator import itemgetter
from textblob import TextBlob
import requests
import justext
import operator
import os
import spacy
from multiprocessing import Pool
import lxml.html
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep
import sys
from pyquery import PyQuery as pq
from urlparse import urlparse
import httplib, sys
import gensim
from gensim import corpora, models
import os
import pandas as pd
import numpy as np
import sys
#sys.path.append('/usr/local/lib/python2.7/dist-packages')
from bs4 import BeautifulSoup   
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import sys
import nltk
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup   
import cPickle
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_curve,auc,classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.grid_search import GridSearchCV

C:\Anaconda2\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [169]:
import cPickle as pickle
with open("all_training_objects", 'rb') as fp:
    dt_kw_v1,dt_v1,dt,vector_features,traditional_features,v_t_features,all_feat = pickle.load(fp)
fp.close()
with open("training1", 'rb') as fp:
    dt_v2 = pickle.load(fp)
fp.close()

In [170]:
#join sitesignals and performance signals (using sitename by normalizing to domain.com)
#vector_features.append('IsProd')
#traditional_features.append('IsProd')
#v_t_features.append('IsProd')
#parseval_features = dt_v2.columns - vector_features

site_signals = pd.read_csv("C:\\Users\\badeepak\Documents\Research\Product Rel\\URLlevel\\site_gl_features.csv",sep="\t",header=None)
site_signals.columns = ['site', 'count_url', 'mean_tags', 'meidan_ptags', 'mean_links', 'avg_words_page', 'mean_candidate_keywords', 'total_keywords', 'total_asins', 'total_GLs', 'max_kscore', 'top_gl_1', 'top_gl_2', 'top_gl_3', 'top_gl_cnt_1', 'top_gl_cnt_2', 'top_gl_cnt_3']
perf_signals = pd.read_csv("C:\\Users\\badeepak\Documents\Research\Product Rel\\URLlevel\\site_ctr_feat_01to04.csv",sep="\t")

perf_signals.columns

perf_sig = perf_signals.dropna().query('no_url > 50').reset_index()
site_sig = site_signals.dropna(subset=['count_url']).query('count_url > 50').reset_index()

dt_v2['sitename'] = [i.replace('http://www.','').replace('https://www.','').replace('www.','') for i in dt_v2['sitename']]
perf_sig['siteName'] = [i.replace('http://www.','').replace('https://www.','').replace('www.','') for i in perf_sig['siteName']]
site_sig['site'] = [i.replace('http://www.','').replace('https://www.','').replace('www.','') for i in site_sig['site']]

tmp = dt_v2.merge(perf_sig,how='left',left_on='sitename',right_on='siteName')
tmp = tmp.merge(site_sig,how='left',left_on='sitename',right_on='site')
dt_v3 = tmp

In [292]:
dt_v3['site_gl_concentration'] = dt_v3['top_gl_cnt_1'] * 1.00/dt_v3['count_url']
dt_v3['site_gl_concentration_top3'] = (dt_v3['top_gl_cnt_1'].fillna(0) + dt_v3['top_gl_cnt_2'].replace('null',np.nan).fillna(0).astype('float') + dt_v3['top_gl_cnt_3'].fillna(0).replace('null',np.nan).fillna(0).astype('float')) * 1.00/dt_v3['count_url']
dt_v3['max_kscore'] = dt_v3['max_kscore_x']
dt_v3['max_kscore_site'] = dt_v3['max_kscore_y']
dt_v3['FR'] = dt_v3['fill'] * 1.00/dt_v3['imp']
#dt_v3[['site','top_GL','top_gl_1','top_gl_cnt_1','count_url']]

In [294]:
dt_v3[['FR','IsProd']]

,FR,IsProd
0,0.258065,Yes
1,0.707167,Yes
2,0.070882,No
3,NaN,Yes
4,NaN,Yes
5,0.781471,No
6,0.781471,No
7,0.698992,Yes
8,0.000000,No
9,0.687327,No


In [254]:
dt_v3.query('az_link_cnt > 0')[['documentId','IsProd']].query('IsProd == \'No\'').count() / dt_v3.query('az_link_cnt > 0')[['documentId','IsProd']].count()

documentId    0.345324
IsProd        0.345324
dtype: float64

In [250]:
print dt_v3.query('az_link_cnt > 0')[['documentId','IsProd']].query('IsProd == \'No\'')

                                            documentId IsProd
13             https://www.engineswapdepot.com/?p=8713     No
51             http://www.dronepeople.club/tag/trafego     No
52             http://www.dronepeople.club/tag/trafego     No
82   https://www.matthewsvolvosite.com/forums/viewt...     No
85   https://www.matthewsvolvosite.com/forums/viewt...     No
93   https://bigclosetr.us/topshelf/search/site/bad...     No
94   https://bigclosetr.us/topshelf/search/site/bad...     No
105  http://juicing-for-health.com/excess-calcium-c...     No
109  http://www.lynsire.com/2016/11/my-law-of-attra...     No
159  http://www.cinematography.com/index.php?showto...     No
166  http://www.cruiseshipdrummer.com/2011/12/stack...     No
185  http://interiorhousing.biz/french-dining-room-...     No
237  http://www.cinematography.com/index.php?showto...     No
243  http://www.modifiedlife.com/1987-ford-bronco-c...     No
255  https://www.50gameslike.com/games-like/wheelma...     No
290     

In [256]:
perf_sig.columns

Index([u'index', u'siteName', u'no_url', u'fill', u'imp', u'clk', u'CTR'], dtype='object')

In [291]:
dt_v3['top_gl_cnt_2'].replace('null',np.nan).fillna(0).astype('float')

0         0.0
1        51.0
2       115.0
3         0.0
4         0.0
5        42.0
6      3716.0
7      3104.0
8         0.0
9        10.0
10      573.0
11      840.0
12       35.0
13      159.0
14       89.0
15        1.0
16        7.0
17     1870.0
18       38.0
19     2116.0
20       47.0
21       36.0
22      840.0
23        0.0
24      481.0
25       98.0
26      840.0
27      840.0
28        0.0
29       82.0
        ...  
893    3104.0
894      14.0
895    3104.0
896    1313.0
897       6.0
898       7.0
899      34.0
900      35.0
901     353.0
902     573.0
903     128.0
904     840.0
905       0.0
906      21.0
907     614.0
908       0.0
909     807.0
910      42.0
911     840.0
912      98.0
913     110.0
914     840.0
915     167.0
916      60.0
917       2.0
918     840.0
919       0.0
920       0.0
921     605.0
922       6.0
Name: top_gl_cnt_2, dtype: float64